In [1]:
import os

import numpy as np
import vtk
from vtk.util.numpy_support import numpy_to_vtk, vtk_to_numpy

import pyfocusr
from pyfocusr.vtk_functions import read_vtk_mesh

In [2]:
location_meshes = '../data/'
target_filename = 'target_mesh_15k.vtk'
source_filename = 'source_mesh_15k.vtk'

target_vtk_mesh = read_vtk_mesh(os.path.join(location_meshes, target_filename))
source_vtk_mesh = read_vtk_mesh(os.path.join(location_meshes, source_filename))

In [3]:
reg = pyfocusr.Focusr(vtk_mesh_target=target_vtk_mesh, 
                      vtk_mesh_source=source_vtk_mesh,
                      n_spectral_features=3,
                      n_extra_spectral=3,
                      get_weighted_spectral_coords=True,
                      list_features_to_calc=['curvature'],
                      rigid_reg_max_iterations=100,
                      non_rigid_alpha=0.001,
                      non_rigid_beta=4,
                      non_rigid_n_eigens=100,
                      non_rigid_max_iterations=300,
                      rigid_before_non_rigid_reg=False,
                      projection_smooth_iterations=40,
                      graph_smoothing_iterations=300,
                      include_points_as_features=True,
                      norm_physical_and_spectral=True,
                      feature_weights=np.diag([1,1]),
                      n_coords_spectral_matching=10000)

Loaded Mesh 1
building adjacency matrix
building degree matrix
starting to get laplacian matrix
beginning eigen decomposition
Loaded spectrum 1
Loaded Mesh 2
building adjacency matrix
building degree matrix
starting to get laplacian matrix
beginning eigen decomposition
Loaded spectrum 2


### View mesh features

In [4]:
# reg.graph_source.view_mesh_features(feature_idx=1)

In [5]:
reg.graph_source.eig_vals

array([-8.72493831e-18,  3.87740912e-26,  2.72996443e-04,  4.43620877e-04,
        5.59590844e-04,  8.95735352e-04])

### View un-ordered/corrected/weighted eigenvectors (spectral coordinates)

#### Source Eigenvector 1

In [6]:
eig_vec = 0

In [7]:
reg.graph_source.view_mesh_eig_vec(eig_vec=eig_vec)

Viewer(geometries=[{'vtkClass': 'vtkPolyData', 'points': {'vtkClass': 'vtkPoints', 'name': '_points', 'numberO…

#### Target Eigenvector 1

In [8]:
reg.graph_target.view_mesh_eig_vec(eig_vec=eig_vec)

Viewer(geometries=[{'vtkClass': 'vtkPolyData', 'points': {'vtkClass': 'vtkPoints', 'name': '_points', 'numberO…

# Notes about comparison: 
Depending on if we include features ('curvature') or not, we will get different results for these meshes. 

If we leave the list at `list_features_to_calc` as an empty list `[]` then: <br>
We note that the color gradient is the same (in the same direction) between the two meshes. However, the colours are flipped. This is because the direction of "positive" can be flipped from one mesh to another. This will (should) be corrected automatically before the registration begins. 

If we include `'curvature'` in `list_features_to_calc` with `['curvature']` then: <br>
We get eigenvectors that align in direction and orientation (at least for the first 3 (0, 1, 2). 

Note, the flipped nature of the spectral coordinates can/will be corrected in the algorithm (if it exists). However, this only works because these are both of the same leg (right - as evident by the large side of the trochlear groove) and will likely faily/break if a left and right leg were analyzed. Therefore, all legs should be flipped to be the same side to "fix" things. 

## Begin Registration
This will print out the registration parameters/results as it goes. <br>
- First it will print out the "pairs" of eigenvectors between the two meshes and which ones on the source were flipped to match the target eigenvectors. 
- Second it will print the rigid registration progress and ultimately the registration "results"/"parameters"
- Third it will print the same for the non-rigid registration progress. 


In [9]:
reg.align_maps()

Eigenvector Sorting Results

The matches for eigenvectors were as follows:
Target	|  Source
     0	|  -0    
     1	|  -1    
     2	|  2     
     3	|  3     
     4	|  4     
     5	|  5     
*Negative source values means those eigenvectors were flipped*

Appending Extra Features to Spectral Coords
Number of features (including spectral) used for registartion: 8

Non-Rigid (Deformable) Registration Beginning

Iteration:1
ML:-258953.680; 	ML change (error):258953.680; 	Sigma^2:     0.023; 	Sigma^2 change:     0.051
[                                                                        ]
Iteration:2
ML:-312452.160; 	ML change (error): 53498.480; 	Sigma^2:     0.011; 	Sigma^2 change:     0.012
[                                                                        ]
Iteration:3
ML:-351284.580; 	ML change (error): 38832.420; 	Sigma^2:     0.004; 	Sigma^2 change:     0.007
[                                                                        ]
Iteration:4
ML:-395821.082; 	ML change 

Iteration:43
ML:-527262.989; 	ML change (error):   252.925; 	Sigma^2:     0.000; 	Sigma^2 change:     0.000
[==========                                                              ]
Iteration:44
ML:-527511.834; 	ML change (error):   248.845; 	Sigma^2:     0.000; 	Sigma^2 change:     0.000
[==========                                                              ]
Iteration:45
ML:-527758.069; 	ML change (error):   246.235; 	Sigma^2:     0.000; 	Sigma^2 change:     0.000
[==========                                                              ]
Iteration:46
ML:-527999.316; 	ML change (error):   241.246; 	Sigma^2:     0.000; 	Sigma^2 change:     0.000
[===========                                                             ]
Iteration:47
ML:       nan; 	ML change (error):       nan; 	Sigma^2:       nan; 	Sigma^2 change:       nan
[===========                                                             ]
Registration Performance Metrics
Time to initialize EM: 1.6878750324249268
Time to ini

IndexError: index 14998 is out of bounds for axis 0 with size 14998

## View the spectral coordinates

In [ ]:
reg.view_aligned_spectral_coords(starting_spectral_coord=0, include_rigid_aligned=True, include_unaligned=True)

In [ ]:
reg.view_meshes_colored_by_spectral_correspondences()

In [ ]:
from itkwidgets import view

In [ ]:
view(point_sets=[reg.smoothed_target_coords, reg.source_projected_on_target])

In [ ]:
view(point_sets=[reg.source_projected_on_target])

In [ ]:
len(np.unique(reg.corresponding_target_idx_for_each_source_pt))

In [ ]:
len(np.unique(reg.corresponding_target_idx_for_each_source_pt))

In [ ]:
reg.view_meshes_colored_by_spectral_correspondences()